In [3]:
import os

%pwd


'c:\\Users\\savit\\Research_Summarizer\\research'

In [4]:
import os
%pwd

'c:\\Users\\savit\\Research_Summarizer\\research'

In [5]:
os.chdir(r"C:\Users\savit\Research_Summarizer")
%pwd

'C:\\Users\\savit\\Research_Summarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationsConfig:
    root_dir: Path
    data_path : Path
    tokenizer_name: Path
    



In [7]:
from src.ResearchSummarizer.constants import *
from src.ResearchSummarizer.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_transformations_config(self) -> DataTransformationsConfig:
        config= self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationsConfig(
            root_dir=config.root_dir,
            data_path= config.data_path,
            tokenizer_name= config.tokenizer_name
        )

        return data_transformation_config

In [13]:
import os
from src.ResearchSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

In [14]:
class DataTransformation:
    def __init__(self, config:DataTransformationsConfig):
        self.config= config
        self.tokenizer= AutoTokenizer.from_pretrained(config.tokenizer_name)
    def tokenize(self, batch):
        inputs = ["summarize: " + doc for doc in batch["article"]]
        model_inputs = self.tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
        labels = self.tokenizer(batch["abstract"], max_length=256, truncation=True, padding="max_length")
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    
    def convert(self):
        dataset= load_from_disk(self.config.data_path)
        dataset_pt = dataset.map(self.tokenize, batched= True)
        dataset_pt.save_to_disk(os.path.join(self.config.root_dir, "scientific_dataset"))
        

In [15]:
config=ConfigurationManager()
data_transformation_config = config.get_data_transformations_config()
data_transformation= DataTransformation(config=data_transformation_config)
data_transformation.convert()


[2025-08-31 23:08:05,404: INFO: common: YAML file loaded successfully: config\config.yaml]
[2025-08-31 23:08:05,406: INFO: common: YAML file loaded successfully: params.yaml]
[2025-08-31 23:08:05,407: INFO: common: ceated dictionary at : artifacts]
[2025-08-31 23:08:05,408: INFO: common: ceated dictionary at : artifacts/data_transformation]


c:\Users\savit\Research_Summarizer\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\savit\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Saving the dataset (1/1 shards): 100%|██████████| 200/200 [00:00<00:00, 12482.68 examples/s]
